In [17]:
# connect to Weaviate
import weaviate

weaviate_client = weaviate.connect_to_local()

In [35]:
# connect to Blogs collection

blogs = weaviate_client.collections.get("WeaviateBlogChunk");

In [19]:
response = blogs.query.hybrid("Product Quantization", limit=2)

stringified_response = ""

for idx, o in enumerate(response.objects):
    stringified_response += f"Search Result {idx+1}:\n"
    for prop in o.properties:
        stringified_response+=f"{prop}:{o.properties[prop]}"
    stringified_response += "\n"

print(stringified_response)

Search Result 1:
content:Let's explore how PQ works. ## Product Quantization
![ann](./img/Ann.png)

If you already know the details behind how Product Quantization works feel free to skip this section!

The main intuition behind Product Quantization is that it adds the concept of segments to the compression function. Basically, to compress the full vector, we will chop it up and operate on segments of it. For example, if we have a 128 dimensional vector we could chop it up into 32 segments, meaning each segment would contain 4 dimensions. Following this segmentation step we compress each segment independently.author:None
Search Result 2:
content:### Product Quantization
While async indexing helped with importing, [product quantization](/developers/weaviate/concepts/vector-quantization#product-quantization) (PQ) was important for alleviating the memory footprint. PQ is a technique used to compress the size of vector embeddings by representing the vectors as centroid ids. The centroid id

In [20]:
# define hybrid search function

def get_search_results(query: str) -> str:
    """Sends a query to Weaviate's Hybrid Search. Parases the response into a {k}:{v} string."""
    
    response = blogs.query.hybrid(query, limit=5)
    
    stringified_response = ""
    for idx, o in enumerate(response.objects):
        stringified_response += f"Search Result: {idx+1}:\n"
        for prop in o.properties:
            stringified_response += f"{prop}:{o.properties[prop]}"
        stringified_response += "\n"
    
    return stringified_response

In [33]:
import ollama
from typing import List, Dict

tools_schema=[{
    'type': 'function',
    'function': {
        'name': 'get_search_results',
        'description': 'Get search results for a provided query.',
        'parameters': {
          'type': 'object',
          'properties': {
            'query': {
              'type': 'string',
              'description': 'The search query.',
            },
          },
          'required': ['query'],
        },
    },
}]

tool_mapping = {
    "get_search_results": get_search_results
}

def ollama_generation_with_tools(user_message: str,
                                 tools_schema: List, tool_mapping: Dict,
                                 model_name: str = "llama3.1") -> str:
    messages=[{
        "role": "user",
        "content": user_message
    }]
    response = ollama.chat(
        model=model_name,
        messages=messages,
        tools=tools_schema
    )
    if not response["message"].get("tool_calls"):
        return response["message"]["content"]
    else:
        for tool in response["message"]["tool_calls"]:
            function_to_call = tool_mapping[tool["function"]["name"]]
            print(f"Calling function {function_to_call}...")
            function_response = function_to_call(tool["function"]["arguments"]["query"])
            messages.append({
                "role": "tool",
                "content": function_response,
            })
    
    final_response = ollama.chat(model=model_name, messages=messages)
    return final_response["message"]["content"]

In [34]:
ollama_generation_with_tools("How is HNSW different from DiskANN?",
                            tools_schema=tools_schema, tool_mapping=tool_mapping)

Calling function <function get_search_results at 0x114749a20>...


"Based on the search results provided, it appears that HNSW (Hierarchical Navigable Small World) is different from DiskANN in the following ways:\n\n1. **Implementation**: While both Vamana and HNSW are ANN algorithms used for vector indexing, Vamana is an implementation of DiskANN, whereas HNSW has its own highly optimized version implemented by Weaviate.\n2. **Performance**: Both Vamana (DiskANN) and the Weaviate-implemented HNSW achieve comparable in-memory results, indicating that both are effective for vector search tasks.\n3. **Challenges**: The implementation of DiskANN on disk-based vectors poses several challenges, including achieving good performance while maintaining excellent database UX.\n\nIn summary, while Vamana (DiskANN) and Weaviate's HNSW share similarities as ANN algorithms, they differ in their implementation, with HNSW having its own optimized version."